<a href="https://colab.research.google.com/github/yashv04/Matchups/blob/main/Batsman_vs_Bowlers_Matchup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max.colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:

df = pd.read_csv("ball by ball 2024.csv")

# Rename columns for clarity
df.rename(columns={
    'batter': 'batsman',
    'batsman_runs': 'runs'
}, inplace=True)

# Create 'over_ball' column
df['over_ball'] = df['over'] + df['ball'] / 10

# Define match phases
def assign_phase(over):
    if over < 6:
        return 'Powerplay'
    elif over < 15:
        return 'Middle'
    else:
        return 'Death'

df['phase'] = df['over'].apply(assign_phase)


df[['match_id', 'batsman', 'bowler', 'over', 'ball', 'over_ball', 'phase', 'runs', 'is_wicket']].head(1)


,match_id,batsman,bowler,over,ball,over_ball,phase,runs,is_wicket
0,335982,SC Ganguly,P Kumar,0,1,0.1,Powerplay,0,0


In [ ]:
# Group data to create matchup summaries
matchups = df.groupby(['batsman', 'bowler', 'phase']).agg(
    balls_faced=('ball', 'count'),
    runs_scored=('runs', 'sum'),
    dismissals=('is_wicket', 'sum')
).reset_index()

# Calculate derived metrics
matchups['strike_rate'] = (matchups['runs_scored'] / matchups['balls_faced']) * 100
matchups['batting_average'] = matchups.apply(
    lambda row: row['runs_scored'] / row['dismissals'] if row['dismissals'] > 0 else None, axis=1
)

matchups.head(1)

,batsman,bowler,phase,balls_faced,runs_scored,dismissals,strike_rate,batting_average
0,A Ashish Reddy,A Nehra,Death,4,2,0,50.0,NaN


In [ ]:
matchups.to_csv("batsman_bowler_matchups.csv", index=False)